In [1]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt


In [2]:
import wandb

wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)


True

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
    

Running on the GPU


In [4]:
REBUILD_DATA = True


class GRV:
    # class to store training data

    # reading data from pickle
    file = open(r"DATA/total_list_w_price_bf.npy", "rb")
    data = pickle.load(file)
    # seperate out classes from inputs
    raceIDs, inputs, classes, prices, win_price, margins, betfairSP = zip(*data)
    # removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df.fillna(value=-1, inplace=True)
    inputs = inputs_df.values.tolist()
    inputs = [[float(i) for i in j] for j in inputs]

    # data
    training_data = []

    def make_training_data(self):
        excluded = 0
        for i in range(len(self.inputs)):
            if len(self.classes[i]) == 8:
                self.training_data.append(
                    [
                        np.array(self.inputs[i]),
                        np.array(self.classes[i]),
                        np.array(self.prices[i]),
                        np.array(self.margins[i]),
                        np.array(self.betfairSP[i]),
                    ]
                )
            else:
                adjustedList = self.classes[i] + ([8] * (8 - len(self.classes[i])))
                adjustedListP = self.prices[i] + ([0] * (8 - len(self.prices[i])))
                adjustedListM = self.margins[i] + ([100] * (8 - len(self.margins[i])))
                adjustedListSP = self.margins[i] + ([0] * (8 - len(self.betfairSP[i])))
                self.training_data.append(
                    [
                        np.array(self.inputs[i]),
                        np.array(adjustedList),
                        np.array(adjustedListP),
                        np.array(adjustedListM),
                        np.array(adjustedListSP)
                    ]
                )
                if len(adjustedList) != 8:
                    print(adjustedList)
        np.save("training_data.npy", self.training_data)
        print("excluded = ", excluded)


if REBUILD_DATA:
    grv = GRV()
    grv.make_training_data()


c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


excluded =  0


In [5]:
grv.betfairSP

([120.0, 12.0, 440.0, 5.599895556, 5.8, 3.0, nan, 50.0],
 [21.197895273503093,
  1.8815589025241928,
  23.296934,
  6.4,
  6.4,
  56.41416643969713,
  20.262803679636438,
  20.4984229903043],
 [7.4,
  91.03969616041284,
  30.45138480166854,
  1.7995775616815202,
  6.733082163684885,
  20.0,
  14.464201088211992,
  26.508447421144847],
 [nan,
  18.0,
  45.76047081084921,
  nan,
  4.832985005122609,
  3.0359346341097995,
  25.787295672437185,
  7.2620745492959395],
 [3.4,
  16.969517280344277,
  11.408280089,
  32.0,
  2.8291454008706207,
  9.931053784615065,
  27.0,
  26.0],
 [9.2,
  13.705378454,
  15.285044260318394,
  3.8718340105350224,
  12.5,
  15.21603800763725,
  6.085789691736865,
  3.942716832],
 [4.88325705054638,
  27.660661303531377,
  5.699330910589779,
  16.5,
  13.414623930350643,
  14.887840753,
  4.222530257670879,
  8.2],
 [18.0,
  nan,
  nan,
  4.550833096578717,
  8.742283643744887,
  nan,
  38.84057459088042],
 [240.97036823254075,
  nan,
  3.2798251233738847,
  14

In [6]:
softmin = nn.Softmin()

# dataset setup
training_data = grv.training_data

X = torch.Tensor([i[0] for i in training_data])
Y = torch.Tensor([i[1] for i in training_data])
P = torch.Tensor([i[2] for i in training_data])
Y_m = softmin(torch.Tensor([i[3] for i in training_data]))
bfSP = torch.Tensor([i[4] for i in training_data])

# Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)

Y_w = torch.tensor([i for i in Y_w])
X = X.to(device)
Y_w = Y_w.to(device)


Y_m = Y_m.to(device)
P = P.to(device)
bfSP = bfSP.to(device)
bfSP = torch.nan_to_num(bfSP, nan=0)
my_dataset = TensorDataset(X, Y_m, P, bfSP)
my_dataloader = DataLoader(my_dataset)


C:\Users\Nick\AppData\Local\Temp\ipykernel_11596\368173627.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  X = torch.Tensor([i[0] for i in training_data])
C:\Users\Nick\AppData\Local\Temp\ipykernel_11596\368173627.py:9: UserWarning: Implicit dimension choice for softmin has been deprecated. Change the call to include dim=X as an argument.
  Y_m = softmin(torch.Tensor([i[3] for i in training_data]))


In [7]:
config = dict(
    epochs=50,
    classes=8,
    kernels=[16, 32],
    batch_size=128,
    learning_rate=0.0001,
    validation_split=0.1,
    dataset="MNIST",
    architecture="CNN",
    dataset_size=len(training_data),
)

config["dataset_size"]


66930

In [8]:
# have to pass in dataset to get_data (created above)
def make_loader(dataset, config, train=True):
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(config["validation_split"] * dataset_size))
    random_seed = 42
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    if train:
        dataset_sampler = SubsetRandomSampler(train_indices)
        loader = torch.utils.data.DataLoader(
            dataset=dataset,
            batch_size=config["batch_size"],
            shuffle=False,
            pin_memory=False,
            num_workers=0,
            sampler=dataset_sampler,
        )
    else:
        dataset_sampler = SubsetRandomSampler(val_indices)
        loader = torch.utils.data.DataLoader(
            dataset=dataset,
            shuffle=False,
            pin_memory=False,
            num_workers=0,
            sampler=dataset_sampler,
        )

    return loader


In [9]:
def build_network(f1_layer_size, f2_layer_size, dropout):

    # network = nn.Sequential(  # fully-connected, dual hidden layer
    #     nn.Linear(120, f1_layer_size),
    #     nn.ReLU(),
    #     nn.Linear(f1_layer_size, f2_layer_size),
    #     nn.ReLU(),
    #     nn.Linear(f2_layer_size, 8),
    #     nn.Softmax(dim=1),
    # )

    network = nn.Sequential(  # fully-connected, dual hidden layer
        nn.Linear(120, f1_layer_size),
        nn.ReLU(),
        nn.Linear(f1_layer_size, f2_layer_size),
        nn.ReLU(),
        nn.Linear(f2_layer_size, f2_layer_size),
        nn.ReLU(),
        nn.Linear(f2_layer_size, 8),
        nn.Softmax(dim=1),
    )

    # network = nn.Sequential(  # fully-connected, dual hidden layer
    #     nn.Linear(120, f1_layer_size),
    #     nn.ReLU(),
    #     nn.Linear(f1_layer_size, 8),
    #     nn.Softmax(dim=1),
    # )

    # network = nn.Sequential(  # fully-connected, dual hidden layer
    #     nn.Linear(120, f1_layer_size),
    #     nn.ReLU(),
    #     nn.Linear(f1_layer_size, f2_layer_size),
    #     nn.ReLU(),
    #     nn.Linear(f2_layer_size, 8),
    # )


    return network


In [10]:
def custom_MSE(output, target):
    sorts = torch.argsort(target)
    out = sorts.narrow(1,0,3)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    out_first3 = ohe*output
    target_ohe = ohe*target
    loss = torch.sum((1+abs(out_first3-target_ohe))**2)
    # loss = torch.sum(((out_first3-target_ohe))**2)
    return loss
    


In [11]:
x = torch.tensor([[0.8,0.2,0.01,0.01,0.01,0.01,0.01,0.01],[1,8,4,5,2,0.01,0.1,0.2]])
y = torch.tensor([[8,0.2,0.9,1,1,1,1,0.1],[1,8,4,5,2,0.01,0.1, 0.2]])
x_idx = custom_MSE(x, y)
x_idx

tensor(18.7602)

In [12]:
def make(config, dataset):
    # Make the data

    train_loader = make_loader(dataset, config, train=True)
    test_loader = make_loader(dataset, config, train=False)
    # Make the model
    # model = Net().to(device)
    model = build_network(
        config["f1_layer_size"], config["f2_layer_size"], config["dropout"]
    )

    loss_functions = {
        "Huber":nn.HuberLoss(),
        "MSE":nn.MSELoss(),
        "L1":nn.SmoothL1Loss(reduction='sum', beta=0.25),
        "BCE":nn.CrossEntropyLoss(),
        "Custom":custom_MSE,
        "KL":nn.KLDivLoss(reduction='batchmean')
    }
    # Make the loss and optimizer
    #  criterion = nn.NLLLoss()
    loss_f = loss_functions[config['loss']]
    criterion = loss_f
    optimizer = config["optimizer"]

    if optimizer == "sgd":
        optimizer = optim.SGD(
            model.parameters(), lr=config["learning_rate"], momentum=0.9
        )
    elif optimizer == "adam":
        optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    # optimizer = torch.optim.Adam(
    #    model.parameters(), lr=config["learning_rate"])

    return model, train_loader, test_loader, criterion, optimizer


In [13]:
def test(model, test_loader, batch_ct):
    model.eval()
    classL, predL, maxL, correctL, priceP, priceR, bfPriceR, pred_odds, model_outputs = [], [], [], [], [], [], [], [], []
    # Run the model on some test examples
    with torch.no_grad():
        correct, total, max_sum, max_w_sum, profit, bfprofit, bfnotavail = 0, 0, 0, 0, 0,0,0
        value_pick_correct, value_pick_profit = 0, 0
        num_bets = 0
        for images, labels, prices, bfspPrices in test_loader:

            outputs = model(images)
            prices = prices[0,].tolist()
            bfspPrices = bfspPrices[0,].tolist()

            max, predicted = torch.max(outputs.data, 1)
            _, real = torch.max(labels.data, 1)

            prediction = predicted.item()
            real_item = real.item()

            predL.append(prediction)
            maxL.append(max.item())

            total += labels.size(0)
            correct += prediction == real_item

            correctL.append(int(prediction == real_item))
            classL.append(real_item)

            priceR.append(prices[real_item])
            priceP.append(prices[prediction])
            bfPriceR.append(bfspPrices[real_item])
            # print(outputs.data.flatten().tolist())

            predicted_odds = [
                1 / ((x + 10**-7)) for x in outputs.data.flatten().tolist()
            ]

            pred_odds.append(predicted_odds)
            model_outputs.append(outputs.data.flatten().tolist())

            if prices[real_item] > (predicted_odds[real_item] * 1.5):
                value_pick_correct += 1
                value_pick_profit += prices[real_item]

            bets = [x > (y * 1.5) for x, y in zip(prices, predicted_odds)]
            num_bets += sum(bets)

            value_pick_profit += -sum(bets)

            if prediction == real_item:
                max_sum += max
                profit += prices[real_item]
                if bfspPrices[real_item]:
                    bfprofit += bfspPrices[real_item]
                else:
                    bfprofit += prices[real_item]
                    bfnotavail += 1
            else:
                max_w_sum += max

            profit += -1
            bfprofit += -1

            # print(f"{correct=}")

        # print(f"Accuracy of the model on the {total} " +
        #       f"test images: {100 * correct / total}%" +
        #       f"profit: {profit}"+
        #       f"profit: {value_pick_profit}")

        wandb.log(
            {
                "test_accuracy": correct / total,
                "correct_conf": max_sum / correct,
                "incorrect_conf": (max_w_sum) / (total - correct),
                "profit": profit,
                "bfprofit": bfprofit,
                "bfnotavail": bfnotavail,
                "value_pick_roi": value_pick_profit / num_bets,
                "num_bets_per": num_bets / total,
            }
        )

        # logdf = pd.DataFrame(
        #     data={
        #         "class": classL,
        #         "pred": predL,
        #         "max": maxL,
        #         "correct": correctL,
        #         "priceR": priceR,
        #         "priceP": priceP,
        #         "bets": sum(bets),
        #         "pred_odds": pred_odds,
        #         "model_outputs": model_outputs,
        #         "bfodds" : bfPriceR
        #     }
        # )
        # table = wandb.Table(dataframe=logdf)
        # wandb.log({"table_key": table})
        # classCounts = logdf["class"].value_counts()
        # predCounts = logdf["pred"].value_counts()
        # boxplot = logdf.boxplot(column=['priceR'],by='correct')
        print(correct, total)
        # print(classCounts, predCounts)
        # boxplot
        # plt.savefig("boxplot.png")
        # wandb.log({"boxplot":boxplot})

    # Save the model in the exchangeable ONNX format
    # torch.onnx.export(model, images, "model.onnx")
    # wandb.save("model.onnx")


In [14]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    #print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [15]:
def train(model, loader,test_loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0

    raw_inputs = True
    if config['loss'] == "KL":

        for epoch in tqdm(range(config.epochs)):
            for _, (images, labels, _ , _) in enumerate(loader):

                loss = train_batch_lsftmax(images, labels, model, optimizer, criterion, btch_count=batch_ct, raw_inputs=True)
                example_ct +=  len(images)
                batch_ct += 1

                # Report metrics every 25th batch
                if ((batch_ct + 1) % 250) == 0:
                    train_log(loss, example_ct, epoch)

            if epoch %10 ==0:
                test(model,test_loader, epoch)

    else:
        for epoch in tqdm(range(config.epochs)):
            for _, (images, labels, _ , _) in enumerate(loader):

                loss = train_batch(images, labels, model, optimizer, criterion, btch_count=batch_ct, raw_inputs=True)
                example_ct +=  len(images)
                batch_ct += 1

                # Report metrics every 25th batch
                if ((batch_ct + 1) % 250) == 0:
                    train_log(loss, example_ct, epoch)

            if epoch %10 ==0:
                test(model,test_loader, epoch)

def train_batch(images, labels, model, optimizer, criterion, btch_count=0, raw_inputs=True):
    images, labels = images, labels
    

    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels.float())
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss


def train_batch_lsftmax(images, labels, model, optimizer, criterion, btch_count=0, raw_inputs=True):
    images, labels = images, labels
    

    # Forward pass ➡
    outputs = model(images)
    loss = criterion(F.log_softmax(outputs), labels.float())
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [16]:
def model_pipeline(config=None):
    dataset = my_dataset
    # tell wandb to get started
    with wandb.init(project="priced after sweep", config=config):
      # access all HPs through wandb.config, so logging matches execution!
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)

      # make the model, data, and optimization problem
      model, train_loader, test_loader, criterion, optimizer = make(config, dataset)
      model = model.to(device)
      print(model)

      # and use them to train the model
      train(model, train_loader,test_loader, criterion, optimizer, config)

      # and test its final performance
      #test(model, test_loader)

    return model

In [17]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'profit',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric



parameters_dict = {
    'optimizer': {
        'value': 'adam'
        },
    'f1_layer_size': {
        'values': [64,128, 256]
        },
    'f2_layer_size': {
        'values': [64,128,256]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'values': [500]},
    'validation_split': {
        'value': 0.1},
    'loss':{
        # 'values': ['Huber', "MSE", "L1", "BCE", "Custom", 'KL']
        'value': 'KL'
            }
    })

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.0001,
        'max': 0.0002
      },
    'batch_size': {
        'values':[64,128,360]
        # 'values':[4,8,16,32,64,128,360]
      }
    })

import pprint

pprint.pprint(sweep_config)



sweep_config

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'profit'},
 'parameters': {'batch_size': {'values': [64, 128, 360]},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'values': [500]},
                'f1_layer_size': {'values': [64, 128, 256]},
                'f2_layer_size': {'values': [64, 128, 256]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.0002,
                                  'min': 0.0001},
                'loss': {'value': 'KL'},
                'optimizer': {'value': 'adam'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'profit', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adam'},
  'f1_layer_size': {'values': [64, 128, 256]},
  'f2_layer_size': {'values': [64, 128, 256]},
  'dropout': {'values': [0.3, 0.4, 0.5]},
  'epochs': {'values': [500]},
  'validation_split': {'value': 0.1},
  'loss': {'value': 'KL'},
  'learning_rate': {'distribution': 'uniform', 'min': 0.0001, 'max': 0.0002},
  'batch_size': {'values': [64, 128, 360]}}}

In [18]:
CUDA_LAUNCH_BLOCKING=1


normal_config = dict(
    epochs=200000,
    classes=8,
    f1_layer_size=64,
    f2_layer_size=64,
    batch_size=128,
    learning_rate=0.000145,
    optimizer = 'adam',
    loss = 'KL',
    dropout = 0.4, 
    validation_split = 0.2,
    dataset="MNIST",
    architecture="CNN",
    dataset_size = len(training_data))


model = model_pipeline(config=normal_config)

  0%|          | 0/200000 [00:00<?, ?it/s]

{'epochs': 200000, 'classes': 8, 'f1_layer_size': 64, 'f2_layer_size': 64, 'batch_size': 128, 'learning_rate': 0.000145, 'optimizer': 'adam', 'loss': 'KL', 'dropout': 0.4, 'validation_split': 0.2, 'dataset': 'MNIST', 'architecture': 'CNN', 'dataset_size': 66930}
200000
{'epochs': 200000, 'classes': 8, 'f1_layer_size': 64, 'f2_layer_size': 64, 'batch_size': 128, 'learning_rate': 0.000145, 'optimizer': 'adam', 'loss': 'KL', 'dropout': 0.4, 'validation_split': 0.2, 'dataset': 'MNIST', 'architecture': 'CNN', 'dataset_size': 66930}
Sequential(
  (0): Linear(in_features=120, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=8, bias=True)
  (7): Softmax(dim=1)
)


C:\Users\Nick\AppData\Local\Temp\ipykernel_11596\2849114048.py:66: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = criterion(F.log_softmax(outputs), labels.float())
  0%|          | 1/200000 [00:25<1405:11:49, 25.29s/it]

2234 13386


  0%|          | 11/200000 [01:13<542:04:49,  9.76s/it]

...

  1%|          | 1471/200000 [2:19:20<627:12:15, 11.37s/it]

3261 13386


  1%|          | 1481/200000 [2:20:18<622:16:29, 11.28s/it]

3254 13386


  1%|          | 1490/200000 [2:20:46<193:07:41,  3.50s/it]